In [3]:
import os
import click
import pickle
from tqdm import tqdm
from numba import njit, prange

import numpy as np
import pandas as pd

In [4]:
@njit
def find_gt(sorted_array, value, start=0):
    i = start
    while i < len(sorted_array):
        if sorted_array[i] >= value:
            break
        i += 1
    return i


def process_file(data, N):
    res = []
    for spec in tqdm(data.keys()):
        intra = data[spec]['intra_distance']/16296
        inter = data[spec]['inter_distance']/16296
        ls = line(intra, inter, N)
        ls = list(map(dict, ls))
        for l in ls:
            l['name'] = spec
        res.extend(ls)

    return res


@njit
def line(intra, inter, N):
    res = []

    sorted_intra = np.sort(intra)
    sorted_inter = np.sort(inter)
    n_intra = len(intra)
    n_inter = len(inter)
    intra_idx = 0
    inter_idx = 0
    for i in range(N):
        x = i / N

        intra_idx = find_gt(sorted_intra, x, intra_idx)
        inter_idx = find_gt(sorted_inter, x, inter_idx)

        tp = intra_idx
        tn = n_inter - inter_idx
        fp = inter_idx
        fn = n_intra - intra_idx

        m = {
            'threshold': x,
            'tp': tp,
            'tn': tn,
            'fp': fp,
            'fn': fn,
        }
        res.append(m)
    return res


filter_name_map = {
    'bilateral_filter': 'Bilateral filter',
    'gaussian_filter': 'Gaussian filter',
    'mean_filter': 'Mean filter',
    'median_filter': 'Median filter',
    'non_local_means': 'Non-local means',
    'uniform_noise': 'Uniform noise',
    'gaussian_noise': 'Gaussian noise',
    'cauchy_noise': 'Cauchy noise',
    'laplacian_noise': 'Laplacian noise',
    'snow': 'Snow noise',
    'salt_and_pepper': 'Salt-and-pepper noise',
    'super_filter': 'Comb',
    'super_filter_reverse': 'Comb Reverse',
    'baseline': 'Baseline'
}

In [5]:
base = 'data/compressed'

In [6]:
def convert(scale, N=10000):
    in_path = os.path.join(base, f'{scale}.pkl')
    out_path = os.path.join('data/metrics', f'{scale}.pkl')
    with open(in_path, 'rb') as f:
        data = pickle.load(f)
        r = pd.DataFrame(process_file(data, N))
        r['far'] = r['fp'] / (r['fp'] + r['tn'])
        r['frr'] = r['fn'] / (r['fn'] + r['tp'])
        r['precision'] = r['tp'] / (r['tp'] + r['fp'])
        r['recall'] = r['tp'] / (r['tp'] + r['fn'])
        r['f1'] = (r['precision'] * r['recall']) / (r['precision'] + r['recall'])
        r['accuracy'] = (r['tp'] + r['tn']) / (r['tp'] + r['tn'] + r['fp'] + r['fn'])
        r['filter'] = r['name'].apply(filter_name_map.get)

        r['scale'] = scale

        r.to_pickle(out_path)

In [7]:
N = 1000
compare_base = ['1xP', '1.5xP', '2xP', '5xP', '10xP']
#compare_self = ['1xS', '1.5xS', '2xS', '5xS', '10xS']

compare_base = [convert(c, N) for c in compare_base]
#compare_self = [convert(c, N) for c in compare_self]

100%|██████████| 14/14 [00:10<00:00,  1.35it/s]
